**Q-3.** 

**Imagine you have a dataset where you have different categories of data, Now
you need to find the most similar data to the given data by using any 4 different
similarity algorithms. Now you have to build a model which can find the most similar
data to the given data.
Dataset This is the Dataset You can use this dataset for this question.**

**Ans.**

In [59]:
pip install xgboost


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [61]:
import os
import re
import json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.tree import DecisionTreeClassifier
import nltk
import string
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings('ignore')

In [63]:
with open('News_Category_Dataset_v3.json','r') as f:
    jdata = f.read()
    jdata2  = [json.loads(line) for line in jdata.split('\n') if line]
    data = pd.DataFrame.from_records(jdata2)
    

In [64]:
df = data.copy()
df.head()

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22


In [65]:
display(df['category'].value_counts())

POLITICS          35602
WELLNESS          17945
ENTERTAINMENT     17362
TRAVEL             9900
STYLE & BEAUTY     9814
PARENTING          8791
HEALTHY LIVING     6694
QUEER VOICES       6347
FOOD & DRINK       6340
BUSINESS           5992
COMEDY             5400
SPORTS             5077
BLACK VOICES       4583
HOME & LIVING      4320
PARENTS            3955
THE WORLDPOST      3664
WEDDINGS           3653
WOMEN              3572
CRIME              3562
IMPACT             3484
DIVORCE            3426
WORLD NEWS         3299
MEDIA              2944
WEIRD NEWS         2777
GREEN              2622
WORLDPOST          2579
RELIGION           2577
STYLE              2254
SCIENCE            2206
TECH               2104
TASTE              2096
MONEY              1756
ARTS               1509
ENVIRONMENT        1444
FIFTY              1401
GOOD NEWS          1398
U.S. NEWS          1377
ARTS & CULTURE     1339
COLLEGE            1144
LATINO VOICES      1130
CULTURE & ARTS     1074
EDUCATION       

In [66]:
selected_cat=['POLITICS', 'ENTERTAINMENT', 'U.S. NEWS', 'WORLD NEWS']
df_new=df[['category','short_description']][df['category'].isin(selected_cat)].reset_index(drop=True)
df_new.head()

,category,short_description
0,U.S. NEWS,Health experts said it is too early to predict...
1,U.S. NEWS,He was subdued by passengers and crew when he ...
2,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...
3,U.S. NEWS,The 63-year-old woman was seen working at the ...
4,U.S. NEWS,"""Who's that behind you?"" an anchor for New Yor..."


In [67]:
class_names0=df_new['category'].unique().tolist()
class_names=sorted(class_names0)
class_names

['ENTERTAINMENT', 'POLITICS', 'U.S. NEWS', 'WORLD NEWS']

In [68]:
N=list(range(len(class_names)))
N

[0, 1, 2, 3]

In [69]:
normal_mapping=dict(zip(class_names,N)) 
normal_mapping

{'ENTERTAINMENT': 0, 'POLITICS': 1, 'U.S. NEWS': 2, 'WORLD NEWS': 3}

In [70]:
reverse_mapping=dict(zip(N,class_names))
reverse_mapping

{0: 'ENTERTAINMENT', 1: 'POLITICS', 2: 'U.S. NEWS', 3: 'WORLD NEWS'}

In [71]:
df_new['category']=df_new['category'].map(normal_mapping)
df_new['category'].head()

0    2
1    2
2    2
3    2
4    2
Name: category, dtype: int64

In [74]:
#train, test = train_test_split(data, test_size=0.2, random_state=42)
import nltk
nltk.download('stopwords')

def clean_text(text):
    '''
    Make text lowercase, remove text in square brackets,remove links,
    remove punctuation and remove words containing numbers.
    '''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

## remove_stopwords
stop_words = stopwords.words('english')
more_stopwords = ['u', 'im', 'c']
stop_words = stop_words + more_stopwords

def remove_stopwords(text):
    words = text.split(' ')
    words = [word for word in words if word not in stop_words]
    text = ' '.join(words)
    return text

##  stemm_text
stemmer = nltk.SnowballStemmer("english")
def stemm_text(text):
    text = ' '.join(stemmer.stem(word) for word in text.split(' '))
    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [75]:
## preprocess_data

def preprocess_data(text):
    # Clean puntuation, urls, and so on
    text = clean_text(text)
    # Remove stopwords
    text = ' '.join(word for word in text.split(' ') if word not in stop_words)
    # Stemm all the words in the sentence
    text = ' '.join(stemmer.stem(word) for word in text.split(' '))
    return text

df_new['clean_text'] = df_new['short_description'].apply(preprocess_data)

df_new['clean_text'].head()

0    health expert said earli predict whether deman...
1    subdu passeng crew fled back aircraft confront...
2    ami cooper accus invest firm franklin templeto...
3     woman seen work south carolina store thursday...
4    whos behind anchor new york  ask journalist mi...
Name: clean_text, dtype: object

In [76]:
X = df_new[['clean_text']]
y = df_new[['category']]

X_train,X_test,y_train,y_test=train_test_split(X, y, test_size = 0.2, random_state = 42)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((46112, 1), (11528, 1), (46112, 1), (11528, 1))

In [77]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

#fit_transform for train data
X_train_vec = vectorizer.fit_transform(X_train['clean_text'])

#transform for test data
X_test_vec = vectorizer.transform(X_test['clean_text'])

In [78]:
def evaluate_model(true, predicted):
    accuracy_score1 = accuracy_score(true, predicted)
    return accuracy_score1

models={
    'LogisticRegression':LogisticRegression(max_iter=1000),
    'DecisionTreeClassifier':DecisionTreeClassifier(),
    'RandomForestClassifier':RandomForestClassifier(),
    'XGBClassifier':XGBClassifier()
}

trained_model_list=[]
model_list=[]
accuracy_score_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train_vec,y_train)

    #Make Predictions
    y_pred=model.predict(X_test_vec)
    #print(y_pred.shape)
    #print(y_test.shape)
    accuracy_score1=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print("Accuracy score",accuracy_score1)

    accuracy_score_list.append(accuracy_score1)
    
    print('='*35)
    print('\n')

LogisticRegression
Accuracy score 0.7799271339347675


DecisionTreeClassifier
Accuracy score 0.7082755031228314


RandomForestClassifier
Accuracy score 0.7583275503122832


XGBClassifier
Accuracy score 0.7373351839000694


